In [ ]:
import cv2
import ipywidgets
import threading
from jetracer.nvidia_racecar import NvidiaRacecar
from jetcam.csi_camera import CSICamera
import traitlets
from IPython.display import display
from ipywidgets import Layout, Button, Box
import ipywidgets.widgets as widgets
from jetcam.utils import bgr8_to_jpeg
import cv2

In [2]:
car = NvidiaRacecar()
camera = CSICamera(width=500, height=500, capture_fps=65)

In [3]:
camera.running = True

In [4]:
prediction_widget = ipywidgets.Image(format='jpeg', width=500, height=500)

In [36]:
# can't detect using image capture from camera
# image = camera.value
# prediction_widget.value = bgr8_to_jpeg(image)

# data, bbox, _ = detector.detectAndDecode(image)

In [5]:


# Global variable to control thread execution
qr_reader_running = True

def read_qr_code_from_camera():
    """Read QR code from camera feed."""
    global qr_reader_running
    
    try:
        detect = cv2.QRCodeDetector()

        while qr_reader_running:
            frame = camera.value  # Read a frame from the camera
            prediction = frame.copy()
            
            value, bbox, straight_qrcode = detect.detectAndDecode(frame)

            if value:
                print("QR Code Value:", value)
                if bbox is not None and len(bbox) > 0:
                    x1, y1 = bbox[0][0][0], bbox[0][0][1]
                    x2, y2 = bbox[2][0][0], bbox[2][0][1]
                 
                    prediction = cv2.rectangle(prediction, (x1, y1), (x2, y2), (0, 0, 255), 2)
                    prediction = cv2.putText(prediction, value, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

            prediction_widget.value = bgr8_to_jpeg(prediction)

    except Exception as e:
        print("An error occurred:", str(e))
        return None

# Function to stop the QR code reader thread
def stop_qr_code_reader():
    global qr_reader_running
    qr_reader_running = False

# Function to run read_qr_code_from_camera() in a separate thread
def run_qr_code_reader():
    global qr_reader_thread
    qr_reader_thread = threading.Thread(target=read_qr_code_from_camera)
    qr_reader_thread.start()

# Call the function to start the QR code reader in a separate thread
run_qr_code_reader()

# Call the function to stop the QR code reader thread (wherever you want to stop it)
# stop_qr_code_reader()

In [6]:
display(prediction_widget)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

QR Code Value: Jetracer QR Code
QR Code Value: Jetracer QR Code
QR Code Value: Jetracer QR Code
QR Code Value: Jetracer QR Code
QR Code Value: Jetracer QR Code
QR Code Value: Jetracer QR Code
QR Code Value: Jetracer QR Code
QR Code Value: Jetracer QR Code
QR Code Value: Jetracer QR Code
QR Code Value: Jetracer QR Code
QR Code Value: Jetracer QR Code
QR Code Value: Jetracer QR Code
QR Code Value: Jetracer QR Code
QR Code Value: Jetracer QR Code


In [7]:
stop_qr_code_reader

<function __main__.stop_qr_code_reader()>

In [ ]:
camera.running = False
camera.cap.release()